<a href="https://colab.research.google.com/github/CarolLuca/NER_Legal_Domain_RO/blob/main/Accommodation/accommodation_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U spacy

     |████████████████████████████████| 12.8MB 288kB/s 
     |████████████████████████████████| 1.1MB 36.8MB/s 
     |████████████████████████████████| 460kB 33.9MB/s 
     |████████████████████████████████| 9.1MB 36.5MB/s 
     |████████████████████████████████| 122kB 41.9MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=311fba142dd74f3213246576d1aba01f77b84cda945cee3e43b90b4fdcd66560
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: smart-open 4.2.0
    Uninstallin

In [ ]:
import spacy.cli
spacy.cli.download("ro_core_news_lg")

✔ Download and installation successful
You can now load the package via spacy.load('ro_core_news_lg')


In [ ]:
# Load a spacy model and chekc if it has ner
import spacy
nlp = spacy.load("ro_core_news_lg")
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer']

In [ ]:
# Import and load the spacy model
import spacy
nlp=spacy.load("ro_core_news_lg") 
# Getting the ner component
ner = nlp.get_pipe('ner')

In [ ]:
test_text = "Carol Luca s-a dus la Bucuresti 2021, fara andreea ca sa critice ordonanta guvernului nr. 25 la parlamentul european"
doc = nlp(test_text)
print("Entities in '%s'" % test_text)
for ent in doc.ents:
  print(ent)

Entities in 'Carol Luca s-a dus la Bucuresti 2021, fara andreea ca sa critice ordonanta guvernului nr. 25 la parlamentul european'
Carol Luca
Bucuresti
25
parlamentul european


In [ ]:
# Add the new label to ner
ner.add_label("FOOD")
# Resume training
optimizer = nlp.resume_training()
move_names = list(ner.move_names)
# List of pipes you want to train
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
# List of pipes which should remain unaffected in training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [ ]:
# Importing requirements
from spacy.util import minibatch, compounding
from spacy.tokens import Doc
from spacy.training import Example
import random

# Begin training by disabling other pipeline components
with nlp.disable_pipes(*other_pipes) :

  sizes = compounding(1.0, 4.0, 1.001)
  # Training for 30 iterations     
  for itn in range(30):
    # shuffle examples before training
    random.shuffle(TRAIN_DATA)
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=sizes)
    # ictionary to store losses
    losses = {}
    for batch in batches:
      #texts, annotations = zip(*batch)
      for text, annotations in batch:
        docc = nlp.make_doc(text)
        example = Example.from_dict(docc, annotations)
        nlp.update([example],drop=0.35, sgd=optimizer, losses=losses)
        #print("Losses", losses)
      #nlp.update([text],[annotations], drop=0.35, sgd=optimizer, losses=losses)
      # Calling update() over the iteration
      # nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
      #nlp.update(batch)
      #print("Losses", losses)

/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "	Inspectoratul pentru Situații de Urgență  Mihail ..." with entities "[(1, 87, 'ORG')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "	2003" with entities "[(1, 4, 'TIME')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "	5.0

In [ ]:
# Testing the NER

test_text = "Gasan Carol Luca, prin Ordononanta de Urgenta nr. 23, a dat Guvernul in judecata la Bucuresti, 23 august 2003. "
doc = nlp(test_text)
print("Entities in '%s'" % test_text)
for ent in doc.ents:
  print(ent)

In [ ]:
# Output directory
from pathlib import Path
output_dir=Path('C:\\Users\\Carol Luca\\Desktop\\Code\\RACAI')

# Saving the model to the output directory
if not output_dir.exists():
  output_dir.mkdir()
nlp.meta['name'] = 'my_ner'  # rename model
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

# Loading the model from the directory
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
assert nlp2.get_pipe("ner").move_names == move_names
doc2 = nlp2('Gasan Carol Luca, prin Ordononanta de Urgenta nr. 23, a dat Guvernul in judecata la Bucuresti, 23 august 2003.')
for ent in doc2.ents:
  print(ent.label_, ent.text)